In [34]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import hamming
import gudhi as gd
import seaborn as sns
from statistics import mode
import multiprocessing
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import gudhi as gd
from scipy.spatial.distance import hamming
import plotly.graph_objs as go
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
from networkx.utils import not_implemented_for, pairwise
from concurrent.futures import ThreadPoolExecutor, as_completed


<h3>Dos funciones para obtener los data frames de presencia</h3>

In [35]:
def get_info_by_pathway_and_all_genomes_directions(info):
    data_frames={}
    pathways=set()
    directions={}
    genome_directions={}
    for i in range(info.shape[0]):
        #Revisar si se debe separar esto
        #pathway=info.loc[i,0].split('|')[0].strip('\ufeff')
        pathway=info.loc[i,0].split('|')[0]
        if pathway not in directions.keys():
            directions[pathway]=[i]
        else:
            directions[pathway].append(i)
        #Se cambia la primer columna, para tener los genes
        gen_with_number=info.loc[i,0].split('|')[2]
        gen=gen_with_number[0:gen_with_number.rfind('_')]
        info.loc[i,0]=gen
        #Se obtienen los genomas
        genome=info.loc[i,1].split('|')[2]
        if genome not in genome_directions.keys():
            genome_directions[genome]=[i]
        else:
            genome_directions[genome].append(i)

    
    for pathway in directions.keys():
        data_frames[pathway]=info.loc[directions[pathway],:]
    return data_frames,genome_directions

In [36]:
def get_df_by_pathway(info_by_pathway_and_genomes_directions):
    info_by_pathway=info_by_pathway_and_genomes_directions[0]
    genomes_directions=info_by_pathway_and_genomes_directions[1]


    df_by_pathway={}
    df_by_pathway_drop_duplicates={}
    representative_genomes_by_pathway={}

    for path in info_by_pathway.keys():
        info_path=info_by_pathway[path]
        genes=set(info_by_pathway[path][0].values)
        df=pd.DataFrame(index=genomes_directions.keys(),columns=list(genes))
        df.index.name=path
        
        for gen in genes:
            info_path_and_gen=info_path[info_path[0]==gen]
            for genom in genomes_directions.keys():
                direction=set(genomes_directions[genom]) & set(info_path_and_gen.index) 
                genomes_per_gen=info_path_and_gen.loc[list(direction)]
                uniq_genomes_per_gen=genomes_per_gen[1].drop_duplicates()
                df.loc[genom,gen]=uniq_genomes_per_gen.shape[0]

        df.sort_index(inplace=True)

        df_by_pathway[path]=df
        df_by_pathway_drop_duplicates[path]=df.drop_duplicates()
        
        representative_genomes_by_pathway[path]=get_respresentants_drop_duplicated_df(df)

    return df_by_pathway, df_by_pathway_drop_duplicates, representative_genomes_by_pathway

In [37]:
def get_respresentants_drop_duplicated_df(df):
    df.sort_index(inplace=True)
    df_drop_duplicates=df.drop_duplicates()
    representants={}
    name=df.index.name
    df_sort_duplic=df.sort_values(by=list(df.columns)).duplicated()
    df_sort_duplic=df_sort_duplic.reset_index()
    representants[df_sort_duplic.loc[0,name]]=[df_sort_duplic.loc[0,name]]
    actual_key=df_sort_duplic.loc[0,name]
        
    for i in range(1,df_sort_duplic.shape[0]):
        bool_i=df_sort_duplic.loc[i,0]
        indice_i=df_sort_duplic.loc[i,name]
        if bool_i==True:
            representants[actual_key].append(indice_i)
                #actual_key.append(indice_i)
        else:
            representants[df_sort_duplic.loc[i,name]]=[df_sort_duplic.loc[i,name]]
            actual_key=indice_i
    return representants

In [38]:
def traduced_df(df,names):
    diccionario=dict(names[[1,2]].values)
    new_df=df.copy()
    df_name=df.index.name
    new_df[df_name]=np.nan
    for indice in new_df.index:
        new_df.loc[indice,df_name]=diccionario[indice]
    new_df=new_df.set_index(df_name)
    #new_df=new_df.drop(column=[df_name])
    return new_df

In [6]:
def traduced_dict_data_frames(dict_df,names):
    traduced_dict={}
    for key,df in dict_df.items():
        traduced_dict[key]=traduced_df(df,names)
    return traduced_dict

In [7]:
def join_dfs(df_by_pathway):
    list(df_by_pathway.values())[0]
    for key in df_by_pathway.keys():
        dfs.append(df_by_pathway[key])
    return pd.concat(dfs, axis=1)
    
    

In [8]:
def get_df_by_genus_pathway(df_by_pathway,names):
    indice=list(df_by_pathway.values())[0].index
    diccionario=dict(names[[1,2]].values)
    genus=pd.Series(diccionario)[indice]
    genus=genus.apply(lambda x: x.split(' ')[0])
    df_genus_pathway={}
    df_by_genus_pathway_drop_duplicates={}
    representative_genomes_by_genus_pathway={}
    
    for path,df in df_by_pathway.items():
        for name_genus in genus.unique():
            name_genus_path=name_genus+'_'+path

            df_genus_path=df[genus==name_genus].copy()
            df_genus_path.index.name=name_genus_path
            
            df_genus_pathway[name_genus_path]=df_genus_path
            df_by_genus_pathway_drop_duplicates[name_genus_path]=df_genus_path.drop_duplicates()
            representative_genomes_by_genus_pathway[name_genus_path]=get_respresentants_drop_duplicated_df(df_genus_path)



    
    return df_genus_pathway,df_by_genus_pathway_drop_duplicates,representative_genomes_by_genus_pathway

<h3>Funciones para dar peso</h3>

In [9]:
def get_mode(df):
    arr=[]
    for i in df.values:
        for j in i:
            if j !=0:
                arr.append(j)
    return mode(arr)

In [10]:
def get_weights(presence):
    mode=get_mode(presence)
    #peso menor que moda
    piv=presence[presence<mode]
    presence[piv>0]=1
    #peso igual a moda
    presence[presence==mode]=2
    #peso mayor que moda
    presence[presence>mode]=3
    return presence

<h2>Funciones para crear matrices de distancia,los complejos simpliciales</h2>


In [11]:
# Let's assume that "population" is a numpy ndarray with your genomes as rows.
def calculate_hamming_matrix(population):
    # Number of genomes
    num_genomes = population.shape[0]
    # Create an empty matrix for Hamming distances
    hamming_matrix = np.zeros((num_genomes, num_genomes), dtype=float)
   # Calculate the Hamming distance between each pair of genomes
    size=num_genomes*(num_genomes-1)//2
    epsilon=0.0001
    increments=(np.random.RandomState(12).rand(size))*epsilon
    for i in range(num_genomes):
        for j in range(i+1, num_genomes):  # j=i+1 to avoid calculating the same distance twice
            # The Hamming distance is multiplied by the number of genes to convert it into an absolute distance
            #distance = hamming(population[i], population[j]) * len(population[i]) +increments[-1]
            distance = hamming(population[i], population[j]) * len(population[i])
            hamming_matrix[i, j] = distance
            hamming_matrix[j, i] = distance  # The matrix is symmetric
            increments=increments[:-1]
    return hamming_matrix

In [12]:
def plot_all_heatmap(complex_pathways):
    for i in complex_pathways.keys():
        gd.plot_persistence_barcode(complex_pathways[i][0],legend=True)
        plt.title(f"{i}" )

In [13]:
def create_complex(distance_matrix2):
    # Create the Rips simplicial complex from the distance matrix
    rips_complex = gd.RipsComplex(distance_matrix=distance_matrix2)
    # Create the simplex tree from the Rips complex with a maximum dimension of 3
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    # Compute the persistence of the simplicial complex
    persistence = simplex_tree.persistence()
    # Return the persistence diagram or barcode
    return persistence, simplex_tree


In [14]:
def get_complex_by_pathways(path_dict):
    complex_path_dict={}
    for path in path_dict.keys():
        complex_path_dict[path]=create_complex(calculate_hamming_matrix(path_dict[path].values))
    return complex_path_dict

In [15]:
def plot_all_bar_code_pathways(complex_pathways):
    for i in complex_pathways.keys():
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
        
        gd.plot_persistence_diagram(persistence=complex_pathways[i][0], axes=axes[0])
        gd.plot_persistence_barcode(persistence=complex_pathways[i][0], axes=axes[1])
        
        # Agregar un título a la figura
        fig.suptitle(f'Pathway: {i}', fontsize=16)

        # Ajustar el espacio entre las figuras
        plt.subplots_adjust(wspace=0.4)  # Cambia el valor según sea necesario
        
        plt.show()



<h2>Funciones para obtener los uno-hoyos</h2>

In [16]:
def minimum_cycle_basis(G, weight=None, total=None):
    """Returns a minimum weight cycle basis for G

    Minimum weight means a cycle basis for which the total weight
    (length for unweighted graphs) of all the cycles is minimum.

    Parameters
    ----------
    G : NetworkX Graph
    weight: string
        name of the edge attribute to use for edge weights

    Returns
    -------
    A list of cycle lists.  Each cycle list is a list of nodes
    which forms a cycle (loop) in G. Note that the nodes are not
    necessarily returned in a order by which they appear in the cycle

    Examples
    --------
    >>> G = nx.Graph()
    >>> nx.add_cycle(G, [0, 1, 2, 3])
    >>> nx.add_cycle(G, [0, 3, 4, 5])
    >>> nx.minimum_cycle_basis(G)
    [[5, 4, 3, 0], [3, 2, 1, 0]]

    References:
        [1] Kavitha, Telikepalli, et al. "An O(m^2n) Algorithm for
        Minimum Cycle Basis of Graphs."
        http://link.springer.com/article/10.1007/s00453-007-9064-z
        [2] de Pina, J. 1995. Applications of shortest path methods.
        Ph.D. thesis, University of Amsterdam, Netherlands

    See Also
    --------
    simple_cycles, cycle_basis
    """
    # We first split the graph in connected subgraphs
    return sum(
        (_min_cycle_basis(G.subgraph(c), weight,total) for c in nx.connected_components(G)),
        [],
    )

def _min_cycle_basis(G, weight,total):
    cb = []
    cont=0
    # We  extract the edges not in a spanning tree. We do not really need a
    # *minimum* spanning tree. That is why we call the next function with
    # weight=None. Depending on implementation, it may be faster as well
    tree_edges = list(nx.minimum_spanning_edges(G, weight=None, data=False))
    chords = G.edges - tree_edges - {(v, u) for u, v in tree_edges}

    # We maintain a set of vectors orthogonal to sofar found cycles
    set_orth = [{edge} for edge in chords]
    while set_orth:
        if cont==total:
            break
        base = set_orth.pop()
        # kth cycle is "parallel" to kth vector in set_orth
        cycle_edges = _min_cycle(G, base, weight)
        cycle_found=[v for u, v in cycle_edges]
        #print(cycle_found)
        if len(cycle_found)>3:
            cb.append(cycle_found)
            cont+=1

        # now update set_orth so that k+1,k+2... th elements are
        # orthogonal to the newly found cycle, as per [p. 336, 1]
        set_orth = [
            (
                {e for e in orth if e not in base if e[::-1] not in base}
                | {e for e in base if e not in orth if e[::-1] not in orth}
            )
            if sum((e in orth or e[::-1] in orth) for e in cycle_edges) % 2
            else orth
            for orth in set_orth
        ]
    return cb


def _min_cycle(G, orth, weight):
    """
    Computes the minimum weight cycle in G,
    orthogonal to the vector orth as per [p. 338, 1]
    Use (u, 1) to indicate the lifted copy of u (denoted u' in paper).
    """
    Gi = nx.Graph()

    # Add 2 copies of each edge in G to Gi.
    # If edge is in orth, add cross edge; otherwise in-plane edge
    for u, v, wt in G.edges(data=weight, default=1):
        if (u, v) in orth or (v, u) in orth:
            Gi.add_edges_from([(u, (v, 1)), ((u, 1), v)], Gi_weight=wt)
        else:
            Gi.add_edges_from([(u, v), ((u, 1), (v, 1))], Gi_weight=wt)

    # find the shortest length in Gi between n and (n, 1) for each n
    # Note: Use "Gi_weight" for name of weight attribute
    spl = nx.shortest_path_length
    lift = {n: spl(Gi, source=n, target=(n, 1), weight="Gi_weight") for n in G}

    # Now compute that short path in Gi, which translates to a cycle in G
    start = min(lift, key=lift.get)
    end = (start, 1)
    min_path_i = nx.shortest_path(Gi, source=start, target=end, weight="Gi_weight")

    # Now we obtain the actual path, re-map nodes in Gi to those in G
    min_path = [n if n in G else n[0] for n in min_path_i]

    # Now remove the edges that occur two times
    # two passes: flag which edges get kept, then build it
    edgelist = list(pairwise(min_path))
    edgeset = set()
    for e in edgelist:
        if e in edgeset:
            edgeset.remove(e)
        elif e[::-1] in edgeset:
            edgeset.remove(e[::-1])
        else:
            edgeset.add(e)

    min_edgelist = []
    for e in edgelist:
        if e in edgeset:
            min_edgelist.append(e)
            edgeset.remove(e)
        elif e[::-1] in edgeset:
            min_edgelist.append(e[::-1])
            edgeset.remove(e[::-1])

    return min_edgelist

In [17]:
def find_all_cycles_whitout_dict(complex_path):
    persistence,simplex_tree=complex_path
    born_and_number = born_filtraton_value_holes(persistence)
    G=nx.Graph()
    born=born_and_number.keys()
    ciclos_dep=set()
    filtration=0
    for simplex, filt in simplex_tree.get_filtration():
        #if len(ciclos_dep)==num_holes:
         #   break
        
        if filtration!=filt and filtration in born:
            number=born_and_number[filtration]
            print('se buscan ciclos en el tiempo', filt,'para')
            ciclos=minimum_cycle_basis(G,total=number)
            for ciclo in ciclos:
                if len(ciclo)>3:
                    print('Se encontró el ciclo',ciclo,'en el tiempo', filtration)
                    ciclos_dep.add(tuple(ciclo))
                    #Se llena el hoyo
                    for i in ciclo:
                        for j in ciclo:
                            G.add_edge(i,j)
                            
            
        filtration=filt
        
        if len(simplex)==2:
            G.add_edge(simplex[0], simplex[1])


    #direc='pseudomonas_'+complex_path+'.txt'
    #f = open(direc, "a")
    #f.write(str(ciclos_dep))
    #f.close()
        

    return ciclos_dep

In [18]:
def born_filtraton_value_holes(persistence):
    born=[]
    for bar in persistence:
        if bar[0]==1:
            born.append(bar[1][0])
            
    born_and_number=set([(x,born.count(x)) for x  in born])

    return dict(born_and_number)

In [19]:
def find_all_cycles(complex_dict):
    complex_path=complex_dict[0]
    persistence,simplex_tree=complex_dict[1]
    born_and_number = born_filtraton_value_holes(persistence)
    G=nx.Graph()
    born=born_and_number.keys()
    ciclos_dep=set()
    filtration=0
    for simplex, filt in simplex_tree.get_filtration():
        #if len(ciclos_dep)==num_holes:
         #   break
        
        if filtration!=filt and filtration in born:
            number=born_and_number[filtration]
            print('se buscan ciclos en el tiempo', filt,'para',complex_path)
            ciclos=minimum_cycle_basis(G,total=number)
            for ciclo in ciclos:
                if len(ciclo)>3:
                    print('Se encontró el ciclo',ciclo,'en el tiempo', filtration,'para',complex_path)
                    ciclos_dep.add(tuple(ciclo))
                    #Se llena el hoyo
                    for i in ciclo:
                        for j in ciclo:
                            G.add_edge(i,j)
                            
            
        filtration=filt
        
        if len(simplex)==2:
            G.add_edge(simplex[0], simplex[1])


    #direc='pseudomonas_'+complex_path+'.txt'
    #f = open(direc, "a")
    #f.write(str(ciclos_dep))
    #f.close()
        

    return [complex_path,ciclos_dep]

In [189]:
def get_holes_by_pathways(complex_by_path,df_by_pathway_drop_duplicate):
    non_empty={}
    empty={}
    
    for path,comp in complex_by_path.items():
        if comp[0][0][0]==1:
            non_empty[path]=comp
        else:
            empty[path]=[]

    result_list=[]
    with ThreadPoolExecutor() as executor:
        futures=[]
        for item in list(non_empty.items()):
            future = executor.submit(find_all_cycles, item)
            futures.append(future)
        for f in futures:
            result_list.append(f.result())
    holes_by_pathway=dict(result_list)
    holes_by_pathway.update(empty)

    holes_by_path=change_vertex_to_code_per_path(holes_by_pathway,df_by_pathway_drop_duplicate)


    
    return holes_by_path

In [172]:
a={2:3}
b={4:6}
a.update(b)

In [188]:
complex_pathways_orint['Glycolysis'][0][0][0]

1

In [126]:
def change_vertex_to_code_per_path(holes_by_pathway,df_by_pathway_drop_duplicate):
    
    new_holes_by_path={}
    for key in holes_by_pathway.keys():

        num_to_name=df_by_pathway_drop_duplicate[key].reset_index()[key]
        new_holes=[]
        for hole in holes_by_pathway[key]:
            hole_with_name=[]
            for vertex in hole:
                hole_with_name.append(num_to_name[vertex])
            new_holes.append(hole_with_name)
        new_holes_by_path[key]=new_holes
    return new_holes_by_path
    

In [166]:
{1:2}

{1: 2}

In [127]:
holes_by_pathway_orint=get_holes_by_pathways(complex_pathways_orint,df_by_pathway_drop_duplicate_orint)

se buscan ciclos en el tiempo 2.0 para Glycolysis
Se encontró el ciclo [1, 12, 6, 5, 4, 3, 2] en el tiempo 1.0 para Glycolysis
se buscan ciclos en el tiempo 3.0 para TCA
se buscan ciclos en el tiempo 3.0 para 3PGA_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [19, 14, 10, 5] en el tiempo 1.0 para ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [19, 8, 6, 5] en el tiempo 1.0 para ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [1, 0, 13, 6] en el tiempo 2.0 para TCA
Se encontró el ciclo [11, 3, 7, 0] en el tiempo 2.0 para 3PGA_AMINOACIDS
Se encontró el ciclo [7, 1, 0, 9] en el tiempo 2.0 para ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para OXALACETATE_AMINOACIDS
Se encontró el ciclo [9, 6, 1, 0] en el tiempo 2.0 para OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 6.0 para OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para

In [60]:
def change_vertex_to_representants(holes_by_pathway,representative_genomes_by_pathway):
    new_holes_by_path={}
    for key in holes_by_pathway.keys():
        representant_to_all_genomes=representative_genomes_by_pathway[key]
        #num_to_name=df_by_pathway_drop_duplicate[key].reset_index()[key]
        new_holes=[]
        for hole in holes_by_pathway[key]:
            hole_with_name=[]
            for vertex in hole:
                vertex_to_all_vertex_names=representant_to_all_genomes[vertex]
                if len(vertex_to_all_vertex_names)==1:
                    hole_with_name.append(vertex)
                else:
                    equivalent_vertex=[]
                    for i in vertex_to_all_vertex_names:
                        equivalent_vertex.append(i)
                    hole_with_name.append(equivalent_vertex)
                

            new_holes.append(hole_with_name)
        new_holes_by_path[key]=new_holes
    return new_holes_by_path

In [92]:
representative_holes_orint=change_vertex_to_representants(holes_by_pathway_orint,representative_genomes_by_pathway_orint)

In [170]:
a={2:3}
a.update({3:4, 6:7})
a

{2: 3, 3: 4, 6: 7}

In [105]:
change_vertex_to_names_per_path(representative_holes_orint,names_orint)['Glycolysis'][0]

[['Serinicoccus sp. CUA-874 CUA-874', 'Ornithinimicrobium sp. na'],
 'Ornithinimicrobium cryptoxanthini JY.X270',
 ['Ornithinimicrobium ciconiae H23M54',
  'Ornithinimicrobium sp. F0845 F0845',
  'Ornithinimicrobium faecis HY1793'],
 'Ornithinimicrobium humiphilum DSM 12362',
 'Ornithinimicrobium avium AMA3305',
 'Serinicoccus sp. CNJ-927 CNJ-927',
 'Ornithinimicrobium sp. CNJ-824 CNJ-824']

In [107]:
representative_holes_orint['Glycolysis'][0]

[['666666.100006', '666666.100040'],
 '666666.100031',
 ['666666.100020', '666666.100030', '666666.100032'],
 '666666.100019',
 '666666.100010',
 '666666.100008',
 '666666.100007']

In [115]:
dict(names_orint[[1,2]].values)['666666.100032']

'Ornithinimicrobium faecis HY1793'

In [ ]:
change_vertex_to_names_per_path()

In [123]:
def change_vertex_to_names_per_path(holes_by_pathway,names):
    code_to_names=dict(names[[1,2]].values)
    new_holes_by_path={}
    for path,holes in holes_by_pathway.items():
        new_holes=[]
        for hole in holes:
            hole_with_name=[]
            for vertex in hole:
                if type(vertex)==str:
                    hole_with_name.append(code_to_names[vertex])
                else:
                    equivalent_vertex=[]
                    for i in vertex:
                        equivalent_vertex.append(code_to_names[i])
                    hole_with_name.append(equivalent_vertex)

            new_holes.append(hole_with_name)
        new_holes_by_path[path]=new_holes
    return new_holes_by_path

In [27]:
def get_dict_names(names):
    dict_code_and_name={}
    for i in names.index:
        dict_code_and_name[names.loc[i,1]]=names.loc[i,2]
    return dict_code_and_name

In [28]:
def change_codes_to_names(codes,dict_code_and_name):
    names=[]
    for code in codes:
        name_per_element=[]
        for element in code:
            name_per_element.append(dict_code_and_name[element])
        names.append(name_per_element)
    return names

In [29]:
def get_hole_names_by_pathway(holes_by_pathway,dict_code_and_name):
    hole_names_by_pathway={}
    for path in holes_by_pathway.keys():
        hole_names_by_pathway[path]=change_codes_to_names(holes_by_pathway[path],dict_code_and_name)
    return hole_names_by_pathway

In [148]:
def get_resum_df(df_by_pathway,representative_genomes_by_pathway,names,complex_by_pathway,holes_by_pathway):
    holes_by_pathway=change_vertex_to_names_per_path(change_vertex_to_representants(holes_by_pathway,representative_genomes_by_pathway),names)
    holes_genus_pathway=holes_per_genus(holes_by_pathway)
    columns=['num_columnas','num_hoyos','ciclos','ciclos por genero']
    resum_df=pd.DataFrame(columns=columns)
    for path in df_by_pathway.keys():
        resum_df.loc[path,'num_columnas']=df_by_pathway[path].shape[1]
        resum_df.loc[path,'num_hoyos']=len(holes_by_pathway[path])
        resum_df.loc[path,'ciclos']=holes_by_pathway[path]
        resum_df.loc[path,'ciclos por genero']=holes_genus_pathway[path]
        duracion=set()
        for hole in complex_by_pathway[path][0]:
            if hole[0]==1:
                duracion.add(hole[1][1]-hole[1][0])
        resum_df.loc[path,'duración hoyos']=str(list(duracion))
    return resum_df

In [150]:
resum=get_resum_df(df_by_genus_pathway_R5P_AMINOACIDS,representative_genomes_by_genus_R5P_AMINOACIDS,names,complex_genus_R5P_AMINOACIDS,holes_R5P_AMINOACIDS)
resum

/opt/conda/envs/TDA/lib/python3.7/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


,num_columnas,num_hoyos,ciclos,ciclos por genero,duración hoyos
Streptosporangium_R5P_AMINOACIDS,10,2,"[[[Streptosporangium sp. caverna CP029711.1, S...","[[[Streptosporangium, Streptosporangium, Strep...",[1.0]
Thermobispora_R5P_AMINOACIDS,10,0,[],[],[]
Nonomuraea_R5P_AMINOACIDS,10,3,[[[Nonomuraea deserti strain KC310 SMKO0100010...,"[[[Nonomuraea, Nonomuraea, Nonomuraea, Nonomur...",[1.0]
Microbispora_R5P_AMINOACIDS,10,1,[[[Microbispora hainanensis strain DSM 45428 V...,"[[[Microbispora, Microbispora], [Microbispora,...",[1.0]
Herbidospora_R5P_AMINOACIDS,10,0,[],[],[]
Microtetraspora_R5P_AMINOACIDS,10,0,[],[],[]
Planomonospora_R5P_AMINOACIDS,10,0,[],[],[]
Planobispora_R5P_AMINOACIDS,10,0,[],[],[]
Thermoactinospora_R5P_AMINOACIDS,10,0,[],[],[]
Sphaerisporangium_R5P_AMINOACIDS,10,0,[],[],[]


In [31]:
def holes_per_genus(holes_by_pathway):
    holes_genus_by_pathway={}
    for path in holes_by_pathway.keys():
        holes_genus=[]
        for hole in holes_by_pathway[path]:
            genus_hole=[]
            for genome in hole:
                if type(genome)==str:
                    #print(genome)
                    genus_hole.append(genome.split(' ')[0])
                else:
                    equivalent_genomes_genus=[]
                    for i in genome:
                        equivalent_genomes_genus.append(i.split(' ')[0])
                    genus_hole.append(equivalent_genomes_genus)
                        
            holes_genus.append(genus_hole)
        holes_genus_by_pathway[path]=list(holes_genus)
    return holes_genus_by_pathway

<h1> ORINT</h1>

In [89]:
info=pd.read_csv('data/pscpOrinth.blast',sep='\t',header=None)
minimum_score=100
info=info[info[11]>=minimum_score]
info.reset_index(drop=True,inplace=True)
info=info.loc[:,0:1]
names_orint=pd.read_csv('data/Orinth.ids',sep='\t',dtype='object',header=None)

In [43]:
info_by_pathway_and_genomes_directions_orint=get_info_by_pathway_and_all_genomes_directions(info)
df_by_pathway_orint,df_by_pathway_drop_duplicate_orint,representative_genomes_by_pathway_orint=get_df_by_pathway(info_by_pathway_and_genomes_directions_orint)
complex_pathways_orint=get_complex_by_pathways(df_by_pathway_drop_duplicate_orint)

In [116]:
info=pd.read_csv('data/pscpStrepto.blast',sep='\t',header=None)
minimum_score=100
info=info[info[11]>=minimum_score]
info.reset_index(drop=True,inplace=True)
info=info.loc[:,0:1]
names=pd.read_csv('data/Strepto.ids',sep='\t',dtype='object',header=None)

In [117]:
info_by_pathway_and_genomes_directions=get_info_by_pathway_and_all_genomes_directions(info)
df_by_pathway,df_by_pathway_drop_duplicate,representative_genomes_by_pathway=get_df_by_pathway(info_by_pathway_and_genomes_directions)
complex_pathways=get_complex_by_pathways(df_by_pathway_drop_duplicate)

In [118]:
df_by_genus_pathway,df_by_genus_pathway_drop_duplicate,representative_genomes_genus_pathway=get_df_by_genus_pathway(df_by_pathway,names)
complex_genus_pathways=get_complex_by_pathways(df_by_genus_pathway_drop_duplicate)

In [144]:
complex_genus_R5P_AMINOACIDS={}
df_by_genus_R5P_AMINOACIDS_drop_duplicate={}
representative_genomes_by_genus_R5P_AMINOACIDS={}
df_by_genus_pathway_R5P_AMINOACIDS={}
for key in complex_genus_pathways.keys():
    if 'R5P_AMINOACIDS' in key:
        df_by_genus_pathway_R5P_AMINOACIDS[key]=df_by_genus_pathway[key]
        complex_genus_R5P_AMINOACIDS[key]=complex_genus_pathways[key]
        df_by_genus_R5P_AMINOACIDS_drop_duplicate[key]=df_by_genus_pathway_drop_duplicate[key]
        representative_genomes_by_genus_R5P_AMINOACIDS[key]=representative_genomes_genus_pathway[key]

In [128]:
holes_R5P_AMINOACIDS=get_holes_by_pathways(complex_genus_R5P_AMINOACIDS,df_by_genus_R5P_AMINOACIDS_drop_duplicate)

se buscan ciclos en el tiempo 2.0 para Streptosporangium_R5P_AMINOACIDS
Se encontró el ciclo [9, 3, 1, 0] en el tiempo 1.0 para Streptosporangium_R5P_AMINOACIDS
Se encontró el ciclo [4, 7, 3, 1] en el tiempo 1.0 para Streptosporangium_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Nonomuraea_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Microbispora_R5P_AMINOACIDS
Se encontró el ciclo [10, 2, 7, 3] en el tiempo 1.0 para Microbispora_R5P_AMINOACIDS
Se encontró el ciclo [5, 10, 2, 1] en el tiempo 1.0 para Nonomuraea_R5P_AMINOACIDS
Se encontró el ciclo [14, 18, 5, 1] en el tiempo 1.0 para Nonomuraea_R5P_AMINOACIDS
Se encontró el ciclo [7, 2, 1, 0] en el tiempo 1.0 para Nonomuraea_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Acrocarpospora_R5P_AMINOACIDS
Se encontró el ciclo [4, 2, 1, 0] en el tiempo 1.0 para Acrocarpospora_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [43, 44, 24, 17] en el tiempo 1.0

In [122]:
df_by_genus_pathway_R5P_AMINOACIDS['Microbispora_R5P_AMINOACIDS']

,ribosephosphate_pyrophosphokinase,histidinol_phosphatase,phosphoribosyl_AMP_cyclohydrolase,imidazoleglycerol_phosphate_dehydratase,phosphoribosyl_ATP_diphosphatase,imidazole_glycerol_phosphate_synthase_H,histidinol_dehydrogenase,ATP_phosphoribosyltransferase,histidinolphosphate_aminotransferase,phosphoribosyl_isomerase_A
Microbispora_R5P_AMINOACIDS,,,,,,,,,,
6666666.1069445,3,1,1,1,1,2,1,1,2,1
6666666.1069448,3,1,1,1,1,2,1,2,1,2
6666666.1069449,2,1,1,1,1,2,1,1,3,1
6666666.1069450,1,1,1,1,1,2,1,1,2,1
6666666.1069456,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6666666.1069938,1,1,0,1,1,2,1,1,2,2
6666666.1069939,2,1,1,1,0,2,1,1,2,1
6666666.1069966,1,1,1,1,1,2,1,1,2,1


In [134]:
df_by_pathway,representative_genomes_by_pathway,names,complex_by_pathway,holes_by_pathway

NameError: name 'complex_by_pathway' is not defined

In [157]:
resum=get_resum_df(df_by_genus_pathway_R5P_AMINOACIDS,representative_genomes_by_genus_R5P_AMINOACIDS,names,complex_genus_R5P_AMINOACIDS,holes_R5P_AMINOACIDS)
resum

/opt/conda/envs/TDA/lib/python3.7/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


,num_columnas,num_hoyos,ciclos,ciclos por genero,duración hoyos
Streptosporangium_R5P_AMINOACIDS,10,2,"[[[Streptosporangium sp. caverna CP029711.1, S...","[[[Streptosporangium, Streptosporangium, Strep...",[1.0]
Thermobispora_R5P_AMINOACIDS,10,0,[],[],[]
Nonomuraea_R5P_AMINOACIDS,10,3,[[[Nonomuraea deserti strain KC310 SMKO0100010...,"[[[Nonomuraea, Nonomuraea, Nonomuraea, Nonomur...",[1.0]
Microbispora_R5P_AMINOACIDS,10,1,[[[Microbispora hainanensis strain DSM 45428 V...,"[[[Microbispora, Microbispora], [Microbispora,...",[1.0]
Herbidospora_R5P_AMINOACIDS,10,0,[],[],[]
Microtetraspora_R5P_AMINOACIDS,10,0,[],[],[]
Planomonospora_R5P_AMINOACIDS,10,0,[],[],[]
Planobispora_R5P_AMINOACIDS,10,0,[],[],[]
Thermoactinospora_R5P_AMINOACIDS,10,0,[],[],[]
Sphaerisporangium_R5P_AMINOACIDS,10,0,[],[],[]


In [118]:
resum.loc['Acrocarpospora_R5P_AMINOACIDS','ciclos'][0]

[['Acrocarpospora catenulata strain H8750 JAHCST010000001.1',
  'Acrocarpospora catenulata strain H8750 NZ_JAHCST010000001.1'],
 ['Acrocarpospora pleiomorpha NBRC 16267 BLAF01000001.1',
  'Acrocarpospora pleiomorpha strain NBRC 16267 NZ_BLAF01000001.1'],
 ['Acrocarpospora macrocephala NBRC 16266 BLAE01000001.1',
  'Acrocarpospora macrocephala strain NBRC 16266 NZ_BLAE01000001.1'],
 ['Acrocarpospora corrugata NBRC 13972 BLAD01000001.1',
  'Acrocarpospora corrugata strain NBRC 13972 NZ_BLAD01000001.1']]

In [158]:
holes_R5P_AMINOACIDS['Nonomuraea_R5P_AMINOACIDS']

[['6666666.1069493', '6666666.1069512', '6666666.1069457', '6666666.1069454'],
 ['6666666.1069532', '6666666.1069633', '6666666.1069493', '6666666.1069454'],
 ['6666666.1069495', '6666666.1069457', '6666666.1069454', '6666666.1069444']]

In [165]:
def get_df_by_pathway_make_hole(df_by_pathway_drop_duplicate,holes_by_pathway):
    df_by_pathway_make_hole={}
    for path,df in df_by_pathway_drop_duplicate.items():
        holes_per_path=[]
        for hole in holes_by_pathway[path]:
            query=[]

            for vertex in hole:
                query.append(vertex)
            
            holes_per_path.append(df.loc[query,])
        df_by_pathway_make_hole[path]=holes_per_path
    return df_by_pathway_make_hole

In [164]:
get_df_by_pathway_make_hole(df_by_genus_pathway_R5P_AMINOACIDS,holes_R5P_AMINOACIDS)['Nonomuraea_R5P_AMINOACIDS'][2]

,ribosephosphate_pyrophosphokinase,histidinol_phosphatase,phosphoribosyl_AMP_cyclohydrolase,imidazoleglycerol_phosphate_dehydratase,phosphoribosyl_ATP_diphosphatase,imidazole_glycerol_phosphate_synthase_H,histidinol_dehydrogenase,ATP_phosphoribosyltransferase,histidinolphosphate_aminotransferase,phosphoribosyl_isomerase_A
Nonomuraea_R5P_AMINOACIDS,,,,,,,,,,
6666666.1069495,3,2,1,1,1,2,1,1,2,1
6666666.1069457,2,2,1,1,1,2,1,1,2,1
6666666.1069454,2,1,1,1,1,2,1,1,2,1
6666666.1069444,3,1,1,1,1,2,1,1,2,1


In [152]:
df_by_genus_pathway_R5P_AMINOACIDS['Streptosporangium_R5P_AMINOACIDS']

,ribosephosphate_pyrophosphokinase,histidinol_phosphatase,phosphoribosyl_AMP_cyclohydrolase,imidazoleglycerol_phosphate_dehydratase,phosphoribosyl_ATP_diphosphatase,imidazole_glycerol_phosphate_synthase_H,histidinol_dehydrogenase,ATP_phosphoribosyltransferase,histidinolphosphate_aminotransferase,phosphoribosyl_isomerase_A
Streptosporangium_R5P_AMINOACIDS,,,,,,,,,,
6666666.1069442,2,2,1,1,1,2,1,1,2,1
6666666.1069452,2,1,1,1,1,2,1,1,2,1
6666666.1069455,3,1,1,1,1,2,1,1,2,1
6666666.1069476,2,1,1,1,1,2,1,1,2,1
6666666.1069480,1,1,1,1,1,2,1,1,2,1
6666666.1069482,2,1,1,1,1,2,1,1,3,1
6666666.1069537,2,1,1,1,1,2,1,1,2,1
6666666.1069545,1,1,1,1,1,2,1,2,3,2
6666666.1069552,2,1,1,1,1,3,1,1,4,1


In [153]:
get_df_by_pathway_make_hole(df_by_genus_R5P_AMINOACIDS_drop_duplicate,holes_R5P_AMINOACIDS)

TypeError: append() takes exactly one argument (0 given)

In [100]:
for i,j in {1:2}.items():
    print(i,j)

1 2


In [95]:
resum.loc['Microbispora_R5P_AMINOACIDS','ciclos'][0]

['Streptosporangiaceae bacterium JOPF424 JASIDX010000164.1',
 'Streptosporangiaceae bacterium JOPF379 JASICE010000027.1',
 'Streptosporangiaceae bacterium JOPF368 JASIBT010000008.1',
 'Streptosporangiaceae bacterium JOPF306 JASHZJ010000847.1']

In [94]:
traduced_df(df_by_genus_pathway_drop_duplicate['Microbispora_R5P_AMINOACIDS'],names)

,imidazoleglycerol_phosphate_dehydratase,ribosephosphate_pyrophosphokinase,imidazole_glycerol_phosphate_synthase_H,phosphoribosyl_AMP_cyclohydrolase,phosphoribosyl_ATP_diphosphatase,phosphoribosyl_isomerase_A,histidinolphosphate_aminotransferase,histidinol_dehydrogenase,histidinol_phosphatase,ATP_phosphoribosyltransferase
Microbispora_R5P_AMINOACIDS,,,,,,,,,,
Microbispora sp. ATCC PTA-5024 KI866522.1,1,3,2,1,1,1,2,1,1,1
Microbispora rosea subsp. rosea strain NRRL B-2630,1,3,2,1,1,2,1,1,1,2
Microbispora rosea subsp. nonnitritogenes strain NRRL B-2631,1,2,2,1,1,1,3,1,1,1
Microbispora sp. NRRL B-24597 JOAF01000001.1,1,1,2,1,1,1,2,1,1,1
Microbispora sp. NBRC 110460 BBOZ01000001.1,0,0,0,0,0,0,0,0,0,0
Microbispora mesophila NBRC 14179 JCM 3151 BBGI01000001.1,1,2,0,0,0,0,1,0,1,2
Microbispora sp. GMKU363 BCBX01000001.1,1,3,2,1,1,1,4,1,1,1
Microbispora sp. GKU 823 MWJN01000001.1,1,2,2,1,1,1,2,1,1,1
Microbispora triticiradicis strain NEAU-HRDPA2-9 C2285 QFZU02000193.1,2,1,4,2,1,3,2,1,1,1


<h1> Strepto</h1>

In [53]:
info=pd.read_csv('data/pscpStrepto.blast',sep='\t',header=None)
minimum_score=100
info=info[info[11]>=minimum_score]
info.reset_index(drop=True,inplace=True)
info=info.loc[:,0:1]
names=pd.read_csv('data/Strepto.ids',sep='\t',dtype='object',header=None)

In [54]:
info_by_pathway_and_genomes_directions=get_info_by_pathway_and_all_genomes_directions(info)
df_by_pathway,df_by_pathway_drop_duplicate,representative_genomes_by_pathway=get_df_by_pathway(info_by_pathway_and_genomes_directions)
complex_pathways=get_complex_by_pathways(df_by_pathway_drop_duplicate)

KeyboardInterrupt: 

In [43]:
change_vertex_to_name_per_path(df_by_pathway_drop_duplicate,diccionario,names,representative_genomes_by_pathway)

{'OXALACETATE_AMINOACIDS': [[['Microbispora rosea subsp. nonnitritogenes strain NRRL B-2631',
    'Microbispora camponoti strain 2C-HV3 JACXRZ010000013.1',
    'Microbispora rosea subsp. rosea NBRC 14044 BOOD01000001.1',
    'Microbispora rosea strain ATCC 12950 FTNI01000083.1',
    'Microbispora rosea subsp. nonnitritogenes strain NRRL B-2631',
    'Microbispora camponoti strain 2C-HV3 NZ_JACXRZ010000013.1',
    'Microbispora rosea subsp. rosea strain NBRC 14044',
    'Microbispora rosea strain ATCC 12950 NZ_FTNI01000083.1'],
   ['Microbispora catharanthi strain CR1-09 VDMA02000010.1',
    'Microbispora sp. H10830 JABBWX010000001.1',
    'Microbispora rosea subsp. rosea strain DSM JACJNS010000001.1',
    'Microbispora catharanthi strain CR1-09 NZ_VDMA02000010.1',
    'Microbispora sp. H10830 NZ_JABBWX010000001.1',
    'Microbispora rosea subsp. rosea strain DSM NZ_JACJNS010000001.1'],
   ['Microbispora sp. H10836 JABBWU010000001.1',
    'Microbispora sp. H13382 JABBXA010000001.1',
   

In [90]:
info=pd.read_csv('data/pscpPseudo.blast',sep='\t',header=None)
minimum_score=100
info=info[info[11]>=minimum_score]
info.reset_index(drop=True,inplace=True)
info=info.loc[:,0:1]
names=pd.read_csv('data/Pseudomonas.ids',sep='\t',dtype='object',header=None)

In [91]:
info_by_pathway_and_genomes_directions=get_info_by_pathway_and_all_genomes_directions(info)
df_by_pathway,df_by_pathway_drop_duplicate,representative_genomes=get_df_by_pathway(info_by_pathway_and_genomes_directions)
complex_pathways=get_complex_by_pathways(df_by_pathway_drop_duplicate)

In [ ]:
join_complex=create_complex(calculate_hamming_matrix(join_df.values))

In [ ]:
join_drop_complex=create_complex(calculate_hamming_matrix(join_df_drop.values))

In [ ]:
join_complex

In [ ]:
gd.plot_persistence_barcode(join_complex[0],legend='Unidos con repeticiones')

In [ ]:
gd.plot_persistence_barcode(join_drop_complex[0],legend='Unidos sin repeticiones')

In [ ]:
for key in df_by_pathway.keys():
    objec=df_by_pathway[key].copy()
    s=objec.drop_duplicates(inplace=False)
    print(df_by_pathway[key].shape[1],s.shape[1],key)


In [ ]:
complex_pathways['Glycolysis'][1].persistence()

In [ ]:
cont=0
vertices=[]
for i in complex_pathways['Glycolysis'][1].get_filtration():
    if i[0]==[0,1]:
        vertices.append(i)

In [ ]:
vertices

In [ ]:
prueba=df_by_pathway['Glycolysis'].drop_duplicates()

In [ ]:
p=create_complex(calculate_hamming_matrix(prueba.values))

In [ ]:
def generar_circulo(radio, num_puntos):
    theta = np.linspace(0, 2 * np.pi, num_puntos)
    x = radio * np.cos(theta)
    y = radio * np.sin(theta)
    return x, y

# Parámetros
num_puntos = 20  # Número de puntos por círculo
radio1 = 3       # Radio del primer círculo
radio2 = 4       # Radio del segundo círculo

# Generar puntos para los círculos
x1, y1 = generar_circulo(radio1, num_puntos)
x2, y2 = generar_circulo(radio2, num_puntos)

In [ ]:
L=[]
for i in range(num_puntos):
    L.append((x1[i],y1[i]))
    L.append((x2[i],y2[i]))
points=np.asanyarray(L)

In [ ]:
x,y=points.T

In [ ]:
plt.scatter(x,y)

In [ ]:
rips=gd.RipsComplex(points=points)
tree=rips.create_simplex_tree(max_dimension=2)
persistance=tree.persistence()

In [ ]:
c=find_all_cycles_whitout_dict((persistance,tree))

In [ ]:
x=visualize_simplicial_complex(tree,4)

In [ ]:
gd.plot_persistence_barcode(persistance)

In [ ]:
def find_all_cycles_whitout_dict(complex_path):
    persistence,simplex_tree=complex_path
    born_and_number = born_filtraton_value_holes(persistence)
    G=nx.Graph()
    born=born_and_number.keys()
    ciclos_dep=set()
    filtration=0
    for simplex, filt in simplex_tree.get_filtration():
        #if len(ciclos_dep)==num_holes:
         #   break
        
        if filtration!=filt and filtration in born:
            number=born_and_number[filtration]
            print('se buscan ciclos en el tiempo', filt,'para')
            ciclos=minimum_cycle_basis(G,total=number)
            for ciclo in ciclos:
                if len(ciclo)>3:
                    print('Se encontró el ciclo',ciclo,'en el tiempo', filtration,'para')
                    ciclos_dep.add(tuple(ciclo))
                    #Se llena el hoyo
                    for i in ciclo:
                        for j in ciclo:
                            G.add_edge(i,j)
                            
            
        filtration=filt
        
        if len(simplex)==2:
            G.add_edge(simplex[0], simplex[1])


    #direc='pseudomonas_'+complex_path+'.txt'
    #f = open(direc, "a")
    #f.write(str(ciclos_dep))
    #f.close()
        

    return ciclos_dep

In [89]:
visualize_simplicial_complex(complex_pathways['R5P_AMINOACIDS'][1],1,'R5P_AMINOACIDS','Pseudomonas')

plotly.graph_objs._figure.Figure

In [83]:
def visualize_simplicial_complex(simplex_tree, filtration_value, path, family,vertex_names=None, save_filename=None, plot_size=1, dpi=600, pos=None):
    G = nx.Graph()
    triangles = []  # List to store triangles (3-nodes simplices)
    
    for simplex, filt in simplex_tree.get_filtration():
        if filt <= filtration_value:
            if len(simplex) == 2:
                G.add_edge(simplex[0], simplex[1])
            elif len(simplex) == 1:
                G.add_node(simplex[0])
            elif len(simplex) == 3:
                triangles.append(simplex)
    
    # Calculate node positions if not provided
    if pos is None:
        pos = nx.spring_layout(G,dim=2)
    
    # Node trace
    x_values, y_values = zip(*[pos[node] for node in G.nodes()])
    node_labels = [vertex_names[node] if vertex_names else str(node) for node in G.nodes()]
    node_trace = go.Scatter(x=x_values, y=y_values, mode='markers+text', hoverinfo='text', marker=dict(size=14), text=node_labels, textposition='top center', textfont=dict(size=14))
        # Edge traces
    edge_traces = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace = go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(width=3, color='rgba(0,0,0,0.5)'))
        edge_traces.append(edge_trace)
    
    # Triangle traces
    triangle_traces = []
    for triangle in triangles:
        x0, y0 = pos[triangle[0]]
        x1, y1 = pos[triangle[1]]
        x2, y2 = pos[triangle[2]]
        triangle_trace = go.Scatter(x=[x0, x1, x2, x0, None], y=[y0, y1, y2, y0, None], fill='toself', mode='lines+markers', line=dict(width=2), fillcolor='rgba(255,0,0,0.2)')
        triangle_traces.append(triangle_trace)
    titulo='Complejo simplicial de la ruta '+path+' en '+family+' para nivel de filtración '+str(filtration_value)
    # Configure the layout of the plot
    layout = go.Layout(showlegend=False, hovermode='closest', xaxis=dict(showgrid=False, zeroline=False, tickfont=dict(size=16, family='Arial, sans-serif')), yaxis=dict(showgrid=False, zeroline=False, tickfont=dict(size=16, family='Arial, sans-serif')),title=dict(
            text=titulo ,
            x=0.5,
            xanchor='center',
            font=dict(size=24, family='Arial, sans-serif')
        ))
    fig = go.Figure(data=edge_traces + triangle_traces + [node_trace], layout=layout)
    
    # Set the figure size
    fig.update_layout(width=plot_size * dpi, height=plot_size * dpi)
    

    # Show the figure
    #fig.write_image('M.png')
    filename=path+'_'+family+'_t_'+str(filtration_value)+'.ht'
    plotly.offline.plot(fig, filename=filename)
    

    return type(fig) 

In [64]:
import plotly
import plotly.express as px

In [52]:
string='''se buscan ciclos en el tiempo 3.0 para Streptosporangium_Glycolysis
Se encontró el ciclo [4, 8, 10, 6] en el tiempo 2.0 para Streptosporangium_Glycolysis
se buscan ciclos en el tiempo 2.0 para Nonomuraea_Glycolysis
se buscan ciclos en el tiempo 2.0 para Microbispora_Glycolysis
se buscan ciclos en el tiempo 2.0 para Herbidospora_Glycolysis
Se encontró el ciclo [2, 4, 1, 0] en el tiempo 1.0 para Herbidospora_Glycolysis
Se encontró el ciclo [22, 2, 17, 9] en el tiempo 1.0 para Microbispora_Glycolysis
Se encontró el ciclo [22, 21, 10, 9] en el tiempo 1.0 para Microbispora_Glycolysis
se buscan ciclos en el tiempo 3.0 para Sphaerisporangium_Glycolysis
Se encontró el ciclo [20, 19, 14, 4] en el tiempo 1.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [32, 14, 19, 11] en el tiempo 1.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [34, 5, 31, 8] en el tiempo 1.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [32, 39, 29, 11] en el tiempo 1.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [32, 14, 4, 12] en el tiempo 1.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [34, 32, 12, 8] en el tiempo 1.0 para Nonomuraea_Glycolysis
se buscan ciclos en el tiempo 3.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [6, 8, 1, 2] en el tiempo 2.0 para Sphaerisporangium_Glycolysis
Se encontró el ciclo [10, 9, 6, 2] en el tiempo 2.0 para Sphaerisporangium_Glycolysis
se buscan ciclos en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
se buscan ciclos en el tiempo 2.0 para Nonomuraea_TCA
se buscan ciclos en el tiempo 2.0 para Microbispora_TCA
Se encontró el ciclo [12, 2, 9, 8] en el tiempo 1.0 para Microbispora_TCA
Se encontró el ciclo [14, 12, 8, 6] en el tiempo 1.0 para Microbispora_TCA
se buscan ciclos en el tiempo 2.0 para Herbidospora_TCA
Se encontró el ciclo [2, 3, 1, 0] en el tiempo 1.0 para Herbidospora_TCA
se buscan ciclos en el tiempo 2.0 para Sphaerisporangium_TCA
Se encontró el ciclo [4, 6, 2, 5] en el tiempo 1.0 para Sphaerisporangium_TCA
se buscan ciclos en el tiempo 3.0 para Streptosporangiaceae_TCA
se buscan ciclos en el tiempo 3.0 para Streptosporangium_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Nonomuraea_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Microbispora_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Sphaerisporangium_3PGA_AMINOACIDS
Se encontró el ciclo [6, 1, 4, 7, 2] en el tiempo 2.0 para Sphaerisporangium_3PGA_AMINOACIDS
Se encontró el ciclo [21, 14, 9, 11] en el tiempo 1.0 para Microbispora_3PGA_AMINOACIDS
Se encontró el ciclo [21, 11, 10, 0] en el tiempo 1.0 para Microbispora_3PGA_AMINOACIDS
Se encontró el ciclo [12, 20, 10, 0] en el tiempo 1.0 para Microbispora_3PGA_AMINOACIDS
Se encontró el ciclo [17, 22, 16, 8] en el tiempo 1.0 para Microbispora_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Microbispora_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [11, 9, 3, 2] en el tiempo 2.0 para Streptosporangium_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Microbispora_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 6.0 para Herbidospora_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [4, 2, 1, 0] en el tiempo 5.0 para Herbidospora_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 4.0 para Sphaerisporangium_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [7, 4, 1, 2] en el tiempo 3.0 para Sphaerisporangium_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [8, 10, 7, 2] en el tiempo 3.0 para Sphaerisporangium_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 6.0 para Streptosporangium_E4P_AMINO_ACIDS
Se encontró el ciclo [9, 39, 3, 1] en el tiempo 1.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [45, 32, 11, 3] en el tiempo 1.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [22, 0, 20, 9] en el tiempo 1.0 para Microbispora_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 15.0 para Microbispora_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [19, 43, 3, 32] en el tiempo 1.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [60, 39, 51, 37] en el tiempo 1.0 para Nonomuraea_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 3.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [11, 17, 10, 6] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [15, 41, 12, 10] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [18, 12, 10, 6] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [16, 4, 10, 6] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [31, 46, 19, 15] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [12, 41, 34, 50, 0, 33, 10] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [16, 13, 9, 2] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [19, 15, 10, 6] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [9, 18, 6, 2] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [30, 31, 15, 10] en el tiempo 1.0 para Nonomuraea_TCA
Se encontró el ciclo [29, 20, 16, 13] en el tiempo 1.0 para Nonomuraea_TCA
se buscan ciclos en el tiempo 3.0 para Nonomuraea_TCA
Se encontró el ciclo [1, 8, 5, 0] en el tiempo 5.0 para Streptosporangium_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 4.0 para Microtetraspora_E4P_AMINO_ACIDS
Se encontró el ciclo [3, 4, 6, 2] en el tiempo 3.0 para Microtetraspora_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 4.0 para Sphaerisporangium_E4P_AMINO_ACIDS
Se encontró el ciclo [9, 10, 7, 5] en el tiempo 3.0 para Sphaerisporangium_E4P_AMINO_ACIDS
Se encontró el ciclo [10, 6, 4, 7] en el tiempo 3.0 para Sphaerisporangium_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 4.0 para Microbispora_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 6.0 para Streptosporangium_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 4.0 para Nonomuraea_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Microbispora_OXALACETATE_AMINOACIDS
Se encontró el ciclo [16, 10, 21, 9] en el tiempo 2.0 para Microbispora_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 8.0 para Microbispora_OXALACETATE_AMINOACIDS
Se encontró el ciclo [8, 9, 2, 0] en el tiempo 5.0 para Streptosporangium_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 7.0 para Streptosporangium_OXALACETATE_AMINOACIDS
Se encontró el ciclo [10, 22, 18, 64, 1] en el tiempo 3.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [10, 34, 35, 60, 1] en el tiempo 3.0 para Nonomuraea_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [54, 79, 55, 25, 15, 74, 1] en el tiempo 2.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [26, 50, 55, 79, 11] en el tiempo 2.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [74, 65, 5, 1] en el tiempo 2.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [26, 65, 5, 1] en el tiempo 2.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [54, 79, 55, 18, 17, 5, 1] en el tiempo 2.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [30, 88, 5, 1] en el tiempo 2.0 para Streptosporangiaceae_TCA
se buscan ciclos en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [1, 13, 2, 3] en el tiempo 6.0 para Streptosporangium_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 5.0 para Sphaerisporangium_OXALACETATE_AMINOACIDS
Se encontró el ciclo [6, 7, 1, 4, 3] en el tiempo 4.0 para Sphaerisporangium_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 5.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [31, 119, 90, 69] en el tiempo 4.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [55, 119, 90, 49] en el tiempo 5.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [19, 4, 2, 8] en el tiempo 2.0 para Microbispora_3PGA_AMINOACIDS
Se encontró el ciclo [11, 23, 8, 19, 10] en el tiempo 2.0 para Microbispora_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Nonomuraea_PPP
Se encontró el ciclo [26, 0, 25, 2] en el tiempo 3.0 para Microbispora_E4P_AMINO_ACIDS
Se encontró el ciclo [24, 15, 26, 7] en el tiempo 3.0 para Microbispora_E4P_AMINO_ACIDS
Se encontró el ciclo [12, 24, 7, 2] en el tiempo 3.0 para Microbispora_E4P_AMINO_ACIDS
Se encontró el ciclo [17, 15, 9, 8] en el tiempo 3.0 para Microbispora_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 2.0 para Herbidospora_PPP
Se encontró el ciclo [4, 2, 1, 0] en el tiempo 1.0 para Herbidospora_PPP
se buscan ciclos en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [16, 34, 15, 1] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [28, 25, 6, 0] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [16, 9, 10, 1] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [6, 11, 2, 0] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [16, 34, 13, 2] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [2, 16, 1, 0] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [6, 3, 1, 0] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [16, 7, 11, 2] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [16, 7, 3, 1] en el tiempo 1.0 para Nonomuraea_PPP
Se encontró el ciclo [28, 32, 10, 1] en el tiempo 1.0 para Nonomuraea_PPP
se buscan ciclos en el tiempo 3.0 para Nonomuraea_PPP
Se encontró el ciclo [96, 77, 82, 13] en el tiempo 1.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [79, 37, 45, 24] en el tiempo 1.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [61, 65, 10, 0] en el tiempo 1.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [113, 82, 13, 11] en el tiempo 1.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [113, 1, 56, 11] en el tiempo 1.0 para Streptosporangiaceae_PPP
se buscan ciclos en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [26, 31, 12, 6] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [44, 21, 22, 10] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [36, 41, 28, 10] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [40, 14, 36, 10] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [34, 25, 6, 1] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [44, 10, 11, 9] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [40, 27, 32, 22] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [28, 40, 13, 1] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [12, 21, 2, 6] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [11, 9, 6, 1] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [28, 10, 11, 1] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [13, 30, 6, 1] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [16, 25, 7, 0] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [28, 12, 6, 1] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [12, 44, 9, 6] en el tiempo 1.0 para Nonomuraea_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [54, 10, 7, 20] en el tiempo 2.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [20, 37, 15, 11, 14, 3] en el tiempo 2.0 para Nonomuraea_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 4.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [69, 47, 112, 55, 119] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [82, 122, 90, 69] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [47, 106, 54, 103, 90, 69] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [65, 45, 119, 31] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [13, 57, 25, 2, 11] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [27, 11, 63, 102] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [45, 8, 5, 119] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [25, 2, 49, 90] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [47, 122, 90, 69] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [55, 12, 13, 5, 119] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [45, 2, 49, 119] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [12, 55, 49, 2, 11] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [24, 79, 57, 13] en el tiempo 6.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [5, 28, 13, 40] en el tiempo 4.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [2, 52, 66, 63, 22] en el tiempo 4.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [10, 41, 61, 15, 65, 38, 5, 28, 58] en el tiempo 4.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [44, 40, 18, 1] en el tiempo 4.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [37, 2, 22, 1] en el tiempo 4.0 para Nonomuraea_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 6.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [48, 103, 20, 29] en el tiempo 4.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [59, 29, 25, 21] en el tiempo 4.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [104, 22, 25, 27] en el tiempo 4.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [30, 10, 17, 1, 5] en el tiempo 4.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [52, 34, 60, 76, 36, 66, 57, 86, 10] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [50, 99, 13, 4] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [81, 20, 33, 4] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [23, 7, 47, 66, 57, 86, 10] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [52, 34, 60, 68, 27, 15, 78, 80, 44, 101, 23] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [29, 23, 13, 4] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [84, 28, 71, 91, 37, 4] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [10, 25, 26, 21, 89, 14, 0] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [81, 37, 41, 31] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [13, 7, 47, 66, 41, 37, 4] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [26, 25, 13, 11] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [25, 13, 23, 10] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [21, 89, 2, 1] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [86, 57, 54, 25] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [81, 18, 13, 4] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [81, 31, 30, 20] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [78, 51, 27, 15] en el tiempo 1.0 para Streptosporangiaceae_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [56, 85, 35, 11] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [12, 18, 28, 111, 82] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [115, 56, 104, 12] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [72, 91, 47, 111, 58] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [115, 99, 100, 17] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [44, 99, 115, 41, 42] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [115, 12, 66, 23] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [109, 22, 89, 111, 58] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [11, 118, 99, 109, 58] en el tiempo 2.0 para Streptosporangiaceae_Glycolysis
se buscan ciclos en el tiempo 4.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [56, 23, 59, 33] en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [54, 35, 45, 32] en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [2, 35, 45, 3] en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [52, 19, 42, 1] en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [52, 12, 3, 1] en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [19, 58, 35, 2] en el tiempo 2.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 4.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [19, 26, 7, 6] en el tiempo 7.0 para Microbispora_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Streptosporangium_PYR_THR_AA
Se encontró el ciclo [15, 9, 2, 0] en el tiempo 1.0 para Streptosporangium_PYR_THR_AA
se buscan ciclos en el tiempo 4.0 para Streptosporangium_PYR_THR_AA
Se encontró el ciclo [17, 12, 11, 9] en el tiempo 3.0 para Streptosporangium_PYR_THR_AA
Se encontró el ciclo [7, 12, 16, 0] en el tiempo 3.0 para Streptosporangium_PYR_THR_AA
se buscan ciclos en el tiempo 3.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [40, 42, 35, 54] en el tiempo 2.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [60, 42, 35, 54] en el tiempo 2.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [15, 34, 60, 46, 21] en el tiempo 2.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [27, 48, 54, 11] en el tiempo 2.0 para Nonomuraea_PYR_THR_AA
se buscan ciclos en el tiempo 4.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [36, 30, 10, 0] en el tiempo 2.0 para Nonomuraea_PPP
Se encontró el ciclo [36, 17, 1, 0] en el tiempo 2.0 para Nonomuraea_PPP
se buscan ciclos en el tiempo 4.0 para Nonomuraea_PPP
Se encontró el ciclo [5, 4, 0, 2] en el tiempo 14.0 para Microbispora_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Microbispora_PYR_THR_AA
Se encontró el ciclo [25, 15, 13, 9] en el tiempo 2.0 para Microbispora_PYR_THR_AA
se buscan ciclos en el tiempo 4.0 para Microbispora_PYR_THR_AA
Se encontró el ciclo [27, 8, 0, 11] en el tiempo 3.0 para Microbispora_PYR_THR_AA
se buscan ciclos en el tiempo 5.0 para Microbispora_PYR_THR_AA
Se encontró el ciclo [27, 28, 0, 10] en el tiempo 2.0 para Nonomuraea_TCA
Se encontró el ciclo [39, 40, 37, 27] en el tiempo 2.0 para Nonomuraea_TCA
Se encontró el ciclo [27, 37, 30, 10] en el tiempo 2.0 para Nonomuraea_TCA
Se encontró el ciclo [27, 44, 15, 10] en el tiempo 2.0 para Nonomuraea_TCA
Se encontró el ciclo [50, 0, 28, 3] en el tiempo 2.0 para Nonomuraea_TCA
se buscan ciclos en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [1, 5, 12, 15] en el tiempo 4.0 para Microbispora_PYR_THR_AA
se buscan ciclos en el tiempo 2.0 para Streptosporangium_R5P_AMINOACIDS
Se encontró el ciclo [9, 3, 1, 0] en el tiempo 1.0 para Streptosporangium_R5P_AMINOACIDS
Se encontró el ciclo [4, 7, 3, 1] en el tiempo 1.0 para Streptosporangium_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Nonomuraea_R5P_AMINOACIDS
Se encontró el ciclo [5, 10, 2, 1] en el tiempo 1.0 para Nonomuraea_R5P_AMINOACIDS
Se encontró el ciclo [14, 18, 5, 1] en el tiempo 1.0 para Nonomuraea_R5P_AMINOACIDS
Se encontró el ciclo [7, 2, 1, 0] en el tiempo 1.0 para Nonomuraea_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Microbispora_R5P_AMINOACIDS
Se encontró el ciclo [10, 2, 7, 3] en el tiempo 1.0 para Microbispora_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Acrocarpospora_R5P_AMINOACIDS
Se encontró el ciclo [4, 2, 1, 0] en el tiempo 1.0 para Acrocarpospora_R5P_AMINOACIDS
se buscan ciclos en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [2, 13, 20, 14] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [107, 123, 109, 66, 11] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [107, 105, 72, 97, 59] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [50, 113, 24, 25] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [96, 112, 55, 11] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [123, 109, 64, 93, 55] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [97, 72, 86, 55] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [51, 93, 17, 27] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [3, 16, 6, 114, 14] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [17, 93, 64, 13, 20, 14] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [24, 113, 10, 20] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [19, 75, 11, 59] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [30, 77, 11, 14] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [107, 123, 41, 92, 59] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [56, 62, 43, 37] en el tiempo 4.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 6.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [43, 44, 24, 17] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [53, 11, 51, 6] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [63, 27, 51, 8] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [51, 30, 17, 8] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [53, 11, 46, 4] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [76, 66, 43, 1] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [8, 51, 6, 1] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [6, 53, 4, 1] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [43, 17, 8, 1] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [4, 5, 3, 1] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [8, 46, 4, 1] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [41, 25, 14, 69] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [31, 61, 25, 41] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [92, 69, 14, 98] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [92, 54, 41, 69] en el tiempo 1.0 para Streptosporangiaceae_R5P_AMINOACIDS
se buscan ciclos en el tiempo 3.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [36, 5, 13, 0] en el tiempo 3.0 para Nonomuraea_PPP
Se encontró el ciclo [6, 38, 5, 2] en el tiempo 2.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [44, 37, 34, 51, 18] en el tiempo 2.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [0, 45, 31, 5] en el tiempo 2.0 para Nonomuraea_Glycolysis
Se encontró el ciclo [39, 41, 59, 1, 26, 20] en el tiempo 3.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [63, 18, 16, 1] en el tiempo 3.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [35, 56, 36, 11] en el tiempo 3.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [26, 12, 24, 20] en el tiempo 3.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [1, 63, 8, 66, 30] en el tiempo 3.0 para Nonomuraea_PYR_THR_AA
se buscan ciclos en el tiempo 5.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [33, 35, 11, 14] en el tiempo 3.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [48, 38, 10, 3] en el tiempo 3.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [4, 25, 0, 22] en el tiempo 3.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [62, 4, 48, 40] en el tiempo 3.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [48, 22, 0, 3] en el tiempo 3.0 para Nonomuraea_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 6.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [19, 20, 44, 10] en el tiempo 2.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [37, 4, 3, 0] en el tiempo 2.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [3, 29, 7, 0] en el tiempo 2.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [42, 20, 29, 7] en el tiempo 2.0 para Nonomuraea_3PGA_AMINOACIDS
Se encontró el ciclo [37, 46, 18, 1] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [51, 3, 7, 58] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [35, 25, 38, 58] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [2, 11, 44, 1] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [65, 26, 23, 10] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [2, 42, 64, 1] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [45, 39, 56, 66] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [7, 3, 39, 56, 13] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [46, 0, 59, 48, 40] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [42, 46, 13, 24] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [6, 16, 59, 48, 41] en el tiempo 5.0 para Nonomuraea_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 7.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [37, 123, 11, 13] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [119, 92, 97, 72] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [76, 92, 22, 100, 20] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [92, 32, 86, 87] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [39, 72, 114, 47, 15, 101, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [119, 72, 36, 100] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [119, 92, 22, 100] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [121, 41, 90, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [101, 109, 62, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [103, 74, 101, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [121, 96, 102, 41] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [97, 72, 78, 112, 46, 19] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [63, 92, 22, 29] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [49, 61, 73, 46] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [39, 109, 62, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [121, 87, 65, 56] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [124, 34, 86, 32, 43] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [37, 123, 55, 39, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [101, 74, 83, 41, 90, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [103, 74, 83, 104, 123, 37] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [45, 43, 92, 97, 39] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [52, 121, 41, 106, 22] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [5, 64, 4, 1] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [73, 61, 86, 34, 46] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [121, 87, 92, 97, 39, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [92, 22, 52, 121, 87] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [59, 29, 30, 50] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [97, 92, 87, 61, 73, 19] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [61, 87, 92, 63, 59, 49] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [4, 109, 101, 15] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [100, 22, 30, 50] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [32, 92, 97, 72, 114, 71, 85] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [17, 64, 4, 13] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [121, 96, 37, 31] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [24, 30, 22, 52, 121, 96, 13] en el tiempo 3.0 para Streptosporangiaceae_PYR_THR_AA
se buscan ciclos en el tiempo 5.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [36, 23, 7, 15] en el tiempo 3.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 5.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [115, 114, 41, 37] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [39, 36, 53, 15] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [112, 105, 49, 24] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [103, 50, 71, 79] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [32, 57, 71, 79, 13] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [99, 88, 33, 24] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [104, 71, 50, 17] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [111, 103, 79, 20] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [99, 113, 53, 15] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [55, 78, 49, 24] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [79, 66, 59, 20] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [27, 58, 107, 4] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [64, 100, 46, 20] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [40, 17, 30, 10] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [85, 38, 3, 9] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [40, 85, 42, 10] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [42, 88, 27, 4] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [1, 57, 97, 46, 11] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [24, 55, 18, 11] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [27, 112, 10, 4] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [55, 100, 46, 24] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [99, 83, 13, 11] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [30, 29, 28, 5] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [1, 18, 3, 9, 2] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [90, 48, 21, 44] en el tiempo 2.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [109, 21, 44, 106] en el tiempo 2.0 para Streptosporangiaceae_PPP
se buscan ciclos en el tiempo 4.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [29, 0, 119, 31] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [38, 21, 60, 97, 55] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [9, 60, 106, 69] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [107, 54, 69, 31] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [2, 118, 23, 61, 106, 90] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [105, 120, 90, 119] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [57, 79, 103, 69] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [97, 60, 20, 0] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [14, 1, 2, 11] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [55, 85, 107, 31] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [99, 67, 12, 13] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [42, 38, 55, 69] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [106, 61, 96, 63, 90] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [74, 38, 56, 64] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [74, 62, 63, 64] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [29, 85, 107, 31] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [63, 96, 38, 56, 64] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [12, 79, 105, 119] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [11, 50, 87, 120, 102] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [85, 34, 47, 55] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [89, 106, 54, 107, 72] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [51, 97, 55, 119] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [71, 122, 62, 15] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [56, 31, 63, 64] en el tiempo 7.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 9.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [74, 72, 28, 56, 12, 51] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [59, 109, 85, 102, 51] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [17, 96, 39, 108, 18] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [104, 33, 103, 32] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [83, 44, 75, 37] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [88, 117, 53, 62] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [55, 98, 20, 3] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [121, 35, 83, 38, 36, 87] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [48, 53, 62, 30, 1, 3] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [108, 39, 44, 75, 114] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [104, 82, 70, 32] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [59, 109, 99, 71, 12, 51] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [116, 33, 104, 94] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [76, 37, 94, 104, 32] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [19, 57, 28, 72, 17, 94] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [13, 98, 55, 1] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [37, 83, 99, 71, 12, 91] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [91, 37, 89, 117, 88, 1] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [36, 38, 83, 37, 91, 51] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [29, 68, 2, 19] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [15, 75, 58, 84, 30, 1] en el tiempo 3.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [87, 102, 55, 1] en el tiempo 3.0 para Streptosporangiaceae_TCA
se buscan ciclos en el tiempo 5.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [60, 2, 1, 5] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [96, 67, 91, 25] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [65, 84, 102, 89, 64, 11] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [31, 45, 107, 115, 82, 105, 13] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [81, 92, 112, 53, 46, 63, 25, 5] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [81, 39, 65, 11] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [115, 84, 65, 92] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [85, 41, 67, 91, 51, 11] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [12, 44, 52, 59, 5] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [73, 51, 11, 5] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [84, 115, 82, 41] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [31, 45, 88, 2, 1, 13] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [105, 41, 85, 11] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [85, 41, 84, 65, 11] en el tiempo 5.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 7.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [76, 78, 58, 2, 6] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [6, 2, 58, 22, 93, 17, 1] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [41, 92, 85, 58, 22] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [81, 15, 90, 1] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [28, 19, 10, 6] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [50, 60, 38, 31, 22, 93, 24] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [76, 78, 84, 13, 1] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [80, 2, 6, 3] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [31, 54, 67, 84, 78, 58, 22] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [96, 68, 36, 39] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [93, 103, 91, 32] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [15, 81, 96, 68] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [98, 65, 93, 103, 14] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [57, 87, 36, 39] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [83, 49, 69, 25] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [90, 15, 42, 45, 85, 58, 2, 6] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [93, 103, 31, 22] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [52, 43, 44, 29] en el tiempo 2.0 para Streptosporangiaceae_R5P_AMINOACIDS
se buscan ciclos en el tiempo 4.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [4, 39, 7, 58] en el tiempo 6.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [49, 48, 41, 21] en el tiempo 6.0 para Nonomuraea_OXALACETATE_AMINOACIDS
Se encontró el ciclo [28, 44, 54, 32] en el tiempo 4.0 para Nonomuraea_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [52, 49, 35, 21] en el tiempo 4.0 para Nonomuraea_PYR_THR_AA
se buscan ciclos en el tiempo 6.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [30, 9, 63, 3] en el tiempo 5.0 para Nonomuraea_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 14.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [69, 59, 60, 43] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [34, 68, 65, 1] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [51, 91, 44, 15] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [10, 36, 104, 32, 1] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [34, 43, 79, 20] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [49, 39, 58, 54] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [72, 59, 14, 0] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [0, 61, 74, 17, 2] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [73, 64, 40, 3] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [49, 62, 61, 54] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [60, 59, 89, 1] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [104, 90, 38, 36] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [21, 45, 51, 60, 1] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [13, 54, 49, 28, 4] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [44, 67, 74, 61, 13] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [40, 42, 48, 3] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [73, 36, 31, 3] en el tiempo 2.0 para Streptosporangiaceae_3PGA_AMINOACIDS
se buscan ciclos en el tiempo 4.0 para Streptosporangiaceae_3PGA_AMINOACIDS
Se encontró el ciclo [48, 96, 20, 65] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [95, 96, 31, 17] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [18, 11, 57, 108, 43] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [102, 107, 32, 42] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [62, 96, 31, 68] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [47, 97, 18, 82] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [107, 51, 62, 96] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [38, 110, 12, 23] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [60, 39, 56, 66] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [50, 45, 18, 82] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [81, 14, 18, 82] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [107, 90, 69, 72] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [105, 122, 102, 86] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [111, 101, 51, 107, 72] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [107, 102, 54, 72] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [89, 65, 25, 16] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [50, 19, 114, 82] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [111, 46, 4, 11] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [84, 75, 62, 68] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [32, 100, 44, 42] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [96, 20, 64, 31] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [63, 58, 22, 43] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [95, 107, 72, 58] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [12, 110, 31, 68] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [110, 48, 96, 31] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [39, 57, 34, 117, 40, 2] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [96, 114, 85, 31] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [92, 64, 85, 44] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [41, 57, 11, 17] en el tiempo 3.0 para Streptosporangiaceae_Glycolysis
se buscan ciclos en el tiempo 5.0 para Streptosporangiaceae_Glycolysis
Se encontró el ciclo [32, 28, 41, 20] en el tiempo 5.0 para Nonomuraea_PYR_THR_AA
Se encontró el ciclo [19, 89, 1, 27] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [37, 32, 108, 17, 56] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [47, 44, 39, 37] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [81, 53, 75, 56] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [123, 116, 19, 59] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [75, 116, 43, 56] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [34, 117, 4, 2] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [90, 49, 39, 42] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [80, 90, 103, 52] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [75, 59, 17, 56] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [75, 62, 101, 85, 11] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [7, 8, 80, 90, 42] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [7, 75, 11, 1] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [7, 75, 56, 42] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [0, 94, 61, 1] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [7, 53, 103, 90, 42] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [108, 126, 2, 27] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [89, 19, 39, 42] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [74, 20, 1, 7, 53, 58] en el tiempo 5.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
se buscan ciclos en el tiempo 7.0 para Streptosporangiaceae_ALPHAKETOGLUTARATE_AMINOACIDS
Se encontró el ciclo [41, 109, 91, 54] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [66, 88, 29, 11] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [15, 36, 117, 95, 20, 31] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [98, 22, 50, 1] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [45, 34, 93, 109, 2] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [23, 21, 55, 90] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [85, 75, 87, 11] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [107, 22, 91, 69] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [89, 108, 74, 8] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [111, 113, 44, 34] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [111, 34, 45, 72] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [123, 19, 63, 64] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [68, 83, 25, 1] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [15, 36, 44, 87, 102] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [95, 73, 97, 20] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [62, 123, 39, 103] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [105, 110, 11, 31] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [110, 89, 50, 11] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [40, 108, 89, 72] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [67, 3, 28, 84, 95, 73, 12] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [87, 109, 2, 102] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [118, 58, 68, 1] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [86, 108, 40, 105] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [98, 109, 2, 1] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [61, 125, 74, 90] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [98, 83, 25, 1] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [65, 72, 40, 105, 31] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [120, 98, 1, 11] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [55, 34, 44, 87, 11] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [89, 53, 113, 111, 72] en el tiempo 8.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
se buscan ciclos en el tiempo 10.0 para Streptosporangiaceae_OXALACETATE_AMINOACIDS
Se encontró el ciclo [121, 71, 37, 20] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [123, 49, 65, 9] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [58, 97, 55, 30] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [106, 71, 79, 46] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [122, 70, 65, 27] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [54, 50, 31, 13] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [99, 121, 25, 5] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [121, 98, 123, 63] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [31, 55, 76, 120, 82] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [109, 110, 78, 31] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [31, 55, 93, 45] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [106, 37, 77, 46] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [40, 83, 113, 5] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [28, 83, 113, 5] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [63, 123, 49, 46] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [123, 35, 32, 72, 61, 121, 63] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [19, 78, 111, 32, 35, 123, 122] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [73, 119, 43, 5] en el tiempo 6.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
se buscan ciclos en el tiempo 8.0 para Streptosporangiaceae_E4P_AMINO_ACIDS
Se encontró el ciclo [38, 113, 50, 20] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [74, 101, 107, 40] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [23, 10, 4, 60] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [21, 50, 30, 13] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [115, 84, 81, 47] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [84, 115, 33, 6] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [82, 124, 45, 62] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [17, 14, 6, 2] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [4, 10, 83, 31] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [23, 115, 47, 39] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [47, 102, 90, 31] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [9, 120, 26, 16, 1] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [122, 41, 43, 46] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [122, 89, 104, 73] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [110, 28, 26, 120, 9] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [33, 117, 97, 121, 1] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [6, 84, 120, 9, 2] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [93, 95, 4, 123] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [36, 113, 50, 67] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [25, 103, 37, 13] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [75, 55, 123, 66] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [75, 8, 107, 66] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [120, 67, 4, 9] en el tiempo 4.0 para Streptosporangiaceae_PYR_THR_AA
se buscan ciclos en el tiempo 6.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [17, 99, 120, 1] en el tiempo 5.0 para Streptosporangiaceae_PYR_THR_AA
Se encontró el ciclo [46, 52, 31, 26, 3] en el tiempo 13.0 para Nonomuraea_E4P_AMINO_ACIDS
Se encontró el ciclo [105, 37, 15, 45] en el tiempo 3.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [40, 94, 60, 24] en el tiempo 3.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [57, 70, 10, 6] en el tiempo 3.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [101, 35, 27, 1] en el tiempo 3.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [58, 71, 39, 1] en el tiempo 3.0 para Streptosporangiaceae_R5P_AMINOACIDS
Se encontró el ciclo [102, 122, 33, 94] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [92, 23, 19, 112] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [70, 118, 46, 31] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [122, 120, 98, 55] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [80, 46, 118, 49] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [109, 33, 94, 51] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [107, 39, 37, 76] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [70, 31, 44, 76] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [71, 9, 0, 2] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [31, 46, 99, 12] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [81, 100, 49, 32] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [110, 46, 118, 49] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [52, 85, 43, 76] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [122, 102, 85, 69] en el tiempo 4.0 para Streptosporangiaceae_TCA
Se encontró el ciclo [50, 108, 48, 21] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [63, 91, 23, 5] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [16, 90, 48, 108, 5] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [75, 106, 59, 11] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [74, 52, 86, 24] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [98, 103, 50, 21] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [87, 72, 57, 55] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [116, 34, 74, 22] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [59, 106, 44, 25] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [23, 52, 30, 5] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [74, 98, 111, 24] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [106, 80, 97, 59] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [74, 87, 55, 24] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [38, 101, 23, 5] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [23, 72, 87, 52] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [70, 91, 72, 87, 22] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [83, 116, 22, 45] en el tiempo 3.0 para Streptosporangiaceae_PPP
Se encontró el ciclo [109, 86, 108, 48] en el tiempo 3.0 para Streptosporangiaceae_PPP'''

In [95]:
renglones=string.split('\n')
renglones_buenos=[]
for i in renglones:
    if ']' in i:
        renglones_buenos.append(i)
diccionario={}
for i in renglones_buenos:
    inicio=i.find('[')
    fin=i.find(']')
    ciclo1=i[inicio+1:fin].split(', ')
    ciclo=[]
    for j in ciclo1:
        ciclo.append(int(j))
    ciclo.sort()
    llave=i.split(' ')[-1]
    if llave not in diccionario.keys():
        diccionario[llave]=[ciclo]
    else:
        diccionario[llave].append(ciclo)

In [96]:
diccionario

{'Glycolysis': [[0, 2, 4, 5], [0, 1, 2, 3]],
 'E4P_AMINO_ACIDS': [[0, 6, 7, 8],
  [0, 2, 4, 13],
  [0, 1, 3, 6],
  [2, 10, 13, 14],
  [0, 4, 5, 6],
  [0, 1, 8, 12]],
 'R5P_AMINOACIDS': [[2, 7, 9, 12],
  [0, 2, 5, 14],
  [0, 3, 15, 24],
  [0, 8, 23, 24],
  [0, 3, 8, 25],
  [0, 1, 2, 10],
  [0, 2, 4, 9],
  [0, 3, 5, 6]],
 'TCA': [[0, 3, 5, 8], [0, 2, 9, 15], [0, 2, 5, 13], [0, 1, 5, 11]],
 '3PGA_AMINOACIDS': [[1, 3, 7, 13],
  [0, 1, 4, 5],
  [2, 6, 16, 18],
  [1, 5, 9, 11],
  [1, 9, 13, 15],
  [1, 3, 8, 14],
  [2, 3, 4, 12],
  [1, 4, 10, 13],
  [1, 3, 5, 6]],
 'PYR_THR_AA': [[6, 14, 15, 81],
  [0, 6, 11, 81],
  [8, 19, 53, 86],
  [30, 37, 45, 64],
  [12, 49, 53, 86],
  [25, 30, 36, 37, 40, 52],
  [12, 26, 31, 58],
  [21, 24, 30, 75],
  [16, 46, 51, 105],
  [56, 61, 76, 82],
  [12, 33, 56, 86],
  [10, 24, 30, 43],
  [12, 22, 82, 86],
  [38, 39, 78, 85],
  [1, 32, 39, 78],
  [38, 50, 60, 85],
  [80, 89, 93, 100],
  [23, 72, 80, 95],
  [21, 63, 66, 79],
  [11, 69, 89, 95],
  [6, 24, 34, 36]